# 1. Regression Code

###### Linear Regression

In [1]:
train_linear <- function(X,y,pars = list()) {
    mod <- lm(y ~ X)
    result <- list()
    result$Yfit = as.matrix(mod$fitted.values)
    result$residuals = as.matrix(mod$residuals)
    result$model = mod
    #for coefficients see list(mod$coef)
    return(result)
}

In [101]:
library(mboost)
train_GAMboost <- function(X,y,pars = list()) #
{
    ## begin old version
    # EXPLANATION: surprisingly, it turned out that this cannot be applied to large p (private discussion with T. Hothorn in Sep 2013)
    # yy <- y
    # dat <- data.frame(cbind(yy,X))
    # gb <- gamboost(yy ~ .,data=dat, baselearner = "bbs")
    ## end old version
    
    ## begin new version
    dat <- as.data.frame(X)
    bl <- lapply(dat, bbs)
    gb <- mboost_fit(bl, y)
    ## end new version
    
    result <- list()
    result$Yfit <- gb$fitted()
    result$residuals <- gb$resid()
    result$model <- gb
    return(result)
}

Loading required package: parallel
Loading required package: stabs
This is mboost 2.9-1. See ‘package?mboost’ and ‘news(package  = "mboost")’
for a complete list of changes.



# 2. Independence Test

In [2]:
library(kernlab)
indtestHsic <- function(x,y,alpha=0.05, pars = list(method = "IncChol")) {    
    if(is.matrix(x)==FALSE){
        x<-as.matrix(x)}
    if(is.matrix(y)==FALSE){
        y<-as.matrix(y)}
    len <- dim(x)[1]
    
    # compute distance matrices
    xnorm<-as.matrix(dist(x,method="euclidean",diag=TRUE,upper=TRUE))
    xnorm<-xnorm^2
    ynorm<-as.matrix(dist(y,method="euclidean",diag=TRUE,upper=TRUE))
    ynorm<-ynorm^2
    
    # choose median heuristic for bandwidth
    if(len>1000) {
        sam <- sample(1:len,1000)
        xhilf<-xnorm[sam,sam]
        yhilf<-ynorm[sam,sam]
    } else {
        xhilf<-xnorm
        yhilf<-ynorm
    }
    
    sigmax<-sqrt(0.5*median(xhilf[lower.tri(xhilf,diag=FALSE)]))
    sigmay<-sqrt(0.5*median(yhilf[lower.tri(yhilf,diag=FALSE)]))
    
    
    if(pars$method == "Exact" || pars$method == "ExactFastTrace") {
        ###
        # Compute GramMat
        ###
        ptm <- proc.time()
        KX <- exp(-xnorm/(2*sigmax^2))
        KY <- exp(-ynorm/(2*sigmay^2))
        timeGramMat <- (proc.time() - ptm)[1]
        
        ###
        # Compute HSIC
        ###
        if(pars$method == "Exact") {
            ptm <- proc.time()
            H<-diag(1,len)-1/len*matrix(1,len,len)
            HSIC <- 1/(len^2)*sum(diag(KX%*%H%*%KY%*%H))
            timeHSIC <- (proc.time() - ptm)[1]
        }
        if(pars$method == "ExactFastTrace") {
            ptm <- proc.time()
            H<-diag(1,len)-1/len*matrix(1,len,len)
            HSIC <- 1/(len^2) * sum((KX - 1/len*(KX%*%rep(1,len))%*%t(rep(1,len)))*t(KY - 1/len*(KY%*%rep(1,len))%*%t(rep(1,len))))
            timeHSIC <- (proc.time() - ptm)[1]
        }
        
        ###
        # Compute Gamma Approximation
        ###
        ptm <- proc.time()
        mux <- (sum(KX)-len)/(len*(len-1))
        muy <- (sum(KY)-len)/(len*(len-1))
        
        mean_h0 <- 1/len*(1+mux*muy-mux-muy)
        var_h0 <- (2*(len-4)*(len-5))/(len*(len-1)*(len-2)*(len-3)) * 1/((len-1)^2)*sum((KX - 1/len*(KX%*%rep(1,len))%*%t(rep(1,len)))*t(KX - 1/len*(KX%*%rep(1,len))%*%t(rep(1,len)))) * 1/((len-1)^2)*sum((KY - 1/len*(KY%*%rep(1,len))%*%t(rep(1,len)))*t(KY - 1/len*(KY%*%rep(1,len))%*%t(rep(1,len))))
        timeGamma <- (proc.time() - ptm)[1]
        
    }
    
    if(pars$method == "IncChol" || pars$method == "IncCholFastTrace") {
        ###
        # Compute GramMat
        ###
        ## incomplete cholesky decomposition
        ptm <- proc.time()
        LX <- inchol(x, kernel="rbfdot", kpar=list(sigma=1/(2*sigmax^2)), tol = 0.0001, maxiter = 300)
        LX <- matrix(LX,nrow=dim(LX)[1], ncol=dim(LX)[2])
        LY <- inchol(y, kernel="rbfdot", kpar=list(sigma=1/(2*sigmay^2)), tol = 0.0001, maxiter = 300)
        LY <- matrix(LY,nrow=dim(LY)[1], ncol=dim(LY)[2])
        LXc <- LX-1/len*(as.matrix(rep(1,len))%*%colSums(LX))
        LYc <- LY-1/len*(as.matrix(rep(1,len))%*%colSums(LY))
        timeGramMat <- (proc.time() - ptm)[1]
        
        ###
        # Compute HSIC
        ###
        if(pars$method == "IncChol") {
            ptm <- proc.time()
            HSIC <- 1/(len^2)*sum(diag((t(LX)%*%LYc)%*%(t(LY)%*%LXc)))
            timeHSIC <- (proc.time() - ptm)[1]
        }
        if(pars$method == "IncCholFastTrace") {
            ptm <- proc.time()
            HSIC <- 1/(len^2)*sum( (t(LX)%*%LYc) * t((t(LY)%*%LXc)))
            timeHSIC <- (proc.time() - ptm)[1]
        }
        
        ###
        # Compute Gamma Approximation
        ###
        ptm <- proc.time()
        mux <- (crossprod(colSums(LX))-len)/(len*(len-1))
        muy <- (crossprod(colSums(LY))-len)/(len*(len-1))
        
        mean_h0 <- 1/len*(1+mux*muy-mux-muy)
        var_h0 <- (2*(len-4)*(len-5))/(len*(len-1)*(len-2)*(len-3))*1/((len-1)^2)*sum(diag((t(LX)%*%LXc)%*%(t(LX)%*%LXc)))*1/((len-1)^2)*sum(diag((t(LY)%*%LYc)%*%(t(LY)%*%LYc)))
        timeGamma <- (proc.time() - ptm)[1]
    }        
    
    a <- (mean_h0^2)/var_h0
    b <- len*var_h0/mean_h0
    critical_value <- qgamma(1-alpha,shape=a,scale=b)
    p_value <- pgamma(len*HSIC,shape=a,scale=b, lower.tail=FALSE)
    resu <- list(statistic = len*HSIC, crit.value = critical_value, p.value = p_value, time = c(timeGramMat,timeHSIC,timeGamma))
    return(resu)
}

# 3. Additive Noise Model Code


In [102]:
test <- function(x, y, reg_model = train_linear, ind_test = indtestHsic, cut_off = 10, verbose = FALSE) {
    xToY <- train_GAMboost(x, y)
    yToX <-  train_GAMboost(y, x)
    xToY.P <- ind_test(xToY$residuals, x)$p.value
    yToX.P <- ind_test(yToX$residuals, y)$p.value
    if (xToY.P > 10 * yToX.P) {
        result <- "X"
    } else if (xToY.P * 10 < yToX.P) {
        result <- "Y"
    } else {
        result <- "NA"
    }

    if (verbose) {
        message("P value for forward is ", xToY.P, ", P value for backword is ", yToX.P, " result is ", ifelse(is.na(result), "Inconclusive", result))
    }
        
    c(result, xToY.P, yToX.P)
}

# 4. Function to Test a Pair

In [4]:
process_file = function(filepath) {
    read.delim(filepath, header = FALSE, sep="", stringsAsFactors = FALSE)    
}

In [42]:
test_pair <- function(data) {
    result <- data.frame(FileName = character(),
                         TestParameter = character(),
                         RegressionModel = character(),
                         Collection = factor(),
                         ForwardPValue = double(),
                         BackwardPValue = double(),
                         Result = factor())
    
    result <- test(
        data[["V1"]],
        data[["V2"]]
    )

    result
}

# 5. Random Sampling

In [34]:
get_random_sample <- function(data, num_rows) {
    data[sample(nrow(data), num_rows), ]
}

# 6. Analysis

In [32]:
file_path = "/Users/mkokkines/Documents/cause_effect_analysis/pairs/pair0043.txt"
data <- process_file(file_path)
print(nrow(data))

[1] 10369


In [38]:
row_sizes = c(100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000)
results <- matrix(list(), nrow = length(row_sizes), ncol = 10)

##### 100 Rows

In [43]:
for (i in 1:10) {
    results[[1, i]] = test_pair(get_random_sample(data, 100))
}

##### 250 Rows

In [44]:
for (i in 1:10) {
    results[[2, i]] = test_pair(get_random_sample(data, 250))
}

##### 500 Rows

In [45]:
for (i in 1:10) {
    results[[3, i]] = test_pair(get_random_sample(data, 500))
}

##### 750 Rows

In [46]:
for (i in 1:10) {
    results[[4, i]] = test_pair(get_random_sample(data, 750))
}

##### 1000 Rows

In [103]:
results1000 <- c()
for (i in 1:100) {
    results1000 <- c(results1000, test_pair(get_random_sample(data, 1000)))
}

In [107]:
sum(results1000 == "X")

[1] 79

##### 1250 Rows

In [48]:
for (i in 1:10) {
    results[[6, i]] = test_pair(get_random_sample(data, 1250))
}

##### 1500 Rows

In [49]:
for (i in 1:10) {
    results[[7, i]] = test_pair(get_random_sample(data, 1500))
}

##### 1750 Rows

In [50]:
for (i in 1:10) {
    results[[8, i]] = test_pair(get_random_sample(data, 1750))
}

##### 2000 Rows

In [51]:
for (i in 1:10) {
    results[[9, i]] = test_pair(get_random_sample(data, 2000))
}

##### 2250 Rows

In [52]:
for (i in 1:10) {
    results[[10, i]] = test_pair(get_random_sample(data, 2250))
}

##### 2500 Rows

In [53]:
for (i in 1:10) {
    results[[11, i]] = test_pair(get_random_sample(data, 2500))
}

##### 2750 Rows

In [54]:
for (i in 1:10) {
    results[[12, i]] = test_pair(get_random_sample(data, 2750))
}

##### 3000 Rows

In [55]:
for (i in 1:10) {
    results[[13, i]] = test_pair(get_random_sample(data, 3000))
}

##### 3250 Rows

In [56]:
for (i in 1:10) {
    results[[14, i]] = test_pair(get_random_sample(data, 3250))
}

##### 3500 Rows

In [57]:
for (i in 1:10) {
    results[[15, i]] = test_pair(get_random_sample(data, 3500))
}

##### 3750 Rows

In [58]:
for (i in 1:10) {
    results[[16, i]] = test_pair(get_random_sample(data, 3750))
}

##### 4000 Rows

In [59]:
for (i in 1:10) {
    results[[17, i]] = test_pair(get_random_sample(data, 4000))
}

##### Determine Accuracy

In [72]:
num_correct = c()

for (i in 1:length(row_sizes)) {
    for (j in 1:10) {
        if (grepl("X", results[i, j][1],  fixed=TRUE)) {
            print(i)
        }
    }
}

[1] 2
[1] 2
[1] 2
[1] 3
[1] 5


In [89]:
print(results[4, ])

[[1]]
[1] "Y"                    "1.11536428285708e-79" "8.17062340197958e-70"

[[2]]
[1] "Y"                    "1.98735682386728e-54" "4.11306050122113e-49"

[[3]]
[1] "Y"                    "2.52937527317185e-54" "8.31757148431584e-48"

[[4]]
[1] "Y"                    "6.34676580252853e-72" "5.43599444615142e-69"

[[5]]
[1] "Y"                    "2.10377632237863e-71" "8.89057113882407e-62"

[[6]]
[1] "Y"                    "6.71061084075854e-75" "8.17607070254791e-69"

[[7]]
[1] "Y"                    "1.42017269547932e-76" "8.33155784613588e-72"

[[8]]
[1] "Y"                    "1.57015568661966e-61" "1.36591089581579e-53"

[[9]]
[1] "Y"                    "9.15511871525159e-53" "3.37672016790063e-49"

[[10]]
[1] "Y"                    "5.47643311604166e-71" "3.49074743749784e-62"

